# European contract pricing with Tree

In [ ]:
import numpy as np
from scipy.optimize import minimize

## Create European contract payoff

In [ ]:
def european_call_payoff(S: float, K: float) -> float:
    return max(S-K, 0.0)

## Create Spot Tree

In [ ]:
def create_spot_tree(spot: float, spot_mult_up: float, spot_mult_down: float, steps: int) -> list[list[float]]:
    previous_level = [spot]
    tree = [previous_level]
    for _ in range(steps):
        new_level = [s * spot_mult_down for s in previous_level]
        new_level += [previous_level[-1] * spot_mult_up]
        tree += [new_level]
        previous_level = new_level
    return tree

In [ ]:
spot = 1
spot_mult_up = 1.2
spot_mult_down = 0.8
steps = 2
spot_tree = create_spot_tree(spot, spot_mult_up, spot_mult_down, steps)
spot_tree

## Create Price Tree

In [ ]:
def create_discounted_price_tree(spot_tree: list[list[float]], discount_factor: float, K: float, diag: int = 0) -> list[list[float]]:
    spot = spot_tree[0][0]
    spot_mult_up = spot_tree[1][-1]
    spot_mult_down = spot_tree[1][0]
    p_up = ((1 / discount_factor - spot_mult_down) /
                   (spot_mult_up - spot_mult_down))
    p_down = 1 - p_up
    steps = len(spot_tree) - 1
    continuation_value_tree = [[np.nan for _ in level] for level in spot_tree]
    if diag > 0:
        print("risk-neutral measure: ")
        print((p_up, p_down))
        # init delta tree
        delta_tree = [[np.nan for _ in level] for level in spot_tree[:-1]] #delta makes no sense for leaves
    # going backwards, payoff is known in leaves
    for i in range(len(spot_tree[-1])):
        spot = spot_tree[-1][i]
        discounted_continuation_value = discount_factor**(steps) * european_call_payoff(spot, K)
        continuation_value_tree[-1][i] = discounted_continuation_value
    for step in range(steps - 1, -1, -1):
        for i in range(len(spot_tree[step])):
            spot = spot_tree[step][i]
            continuation_value_tree[step][i] = p_up * continuation_value_tree[step + 1][i] + \
                                            p_down * continuation_value_tree[step + 1][i + 1]
            if diag > 0:
                delta_tree[step][i] = ((continuation_value_tree[step + 1][i] - continuation_value_tree[step + 1][i + 1]) 
                                       / (spot_tree[step + 1][i] - spot_tree[step + 1][i + 1]))
    if diag > 0:
        print("delta: ")
        print(delta_tree)
    return continuation_value_tree

In [ ]:
discount_factor = 0.95
strike = 1
price_tree = create_discounted_price_tree(spot_tree, discount_factor, strike, 1)
price_tree

In [ ]:
0.44*0.95*0.95

In [ ]:
0.3971 * 0.3684210526315791

## Balanced Tree

In [ ]:
def calcBalancedDownStep(spot_mult_up: float, discount_factor: float) -> (float, float):
    return spot_mult_up - 2 * (spot_mult_up - 1 / discount_factor)

In [ ]:
spot_mult_down_balanced = calcBalancedDownStep(spot_mult_up, discount_factor)
print(spot_mult_down)
spot_tree = create_spot_tree(spot, spot_mult_up, spot_mult_down_balanced, steps)
print(spot_tree)
price_tree = create_discounted_price_tree(spot_tree, discount_factor, strike, 1)
print(price_tree)

## Delta is close to 0.5 for At The Money Forward option

In [ ]:
strike_ATMF = 1/0.95**2
price_tree = create_discounted_price_tree(spot_tree, discount_factor, strike_ATMF, 1)
print(price_tree)

## Calibration

* Spot and price of call option are given (and DF).
* How do we calibrate the tree model to it? i.e., how do we create a model which provides the given price for the given contract?

In [ ]:
strike = 1
spot = 1
call_price = 0.1
discount_factor = 0.95
steps = 1
def price_diff(spot_mult_up: float)->float:
    spot_mult_down_balanced = calcBalancedDownStep(spot_mult_up, discount_factor)
    spot_tree = create_spot_tree(spot, spot_mult_up, spot_mult_down_balanced, steps)
    price_tree = create_discounted_price_tree(spot_tree, discount_factor, strike)
    return abs(call_price - price_tree[0][0])

In [ ]:
res = minimize(price_diff, 2, method='Nelder-Mead', tol=1e-6)

In [ ]:
res

In [ ]:
spot_mult_up = 1.211
spot_mult_down_balanced = calcBalancedDownStep(spot_mult_up, discount_factor)
spot_tree = create_spot_tree(spot, spot_mult_up, spot_mult_down_balanced, steps)
price_tree = create_discounted_price_tree(spot_tree, discount_factor, strike)
price_tree[0][0]

In [ ]:
price_tree